In [39]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mpl_toolkits
%matplotlib inline
import re
import tensorflow as tf
from keras.callbacks import ModelCheckpoint

In [40]:
# Read data
data_w = pd.read_csv("zillow_wake_co.csv")
data_d = pd.read_csv("zillow_durham_co_df.csv")

In [41]:
# Concatenate both dataframes
data = pd.concat([data_w, data_d], axis=0)
data

,props.dateSold,props.propertyType,props.lotAreaValue,props.address,props.variableData.text,props.variableData.type,props.zestimate,props.imgSrc,props.price,props.bedrooms,...,props.daysOnZillow,props.bathrooms,props.livingArea,props.country,props.currency,props.lotAreaUnit,props.hasImage,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,1682665200000,SINGLE_FAMILY,0.920000,"7525 Matherly Dr, Wake Forest, NC 27587",Sold 04/28/2023,RECENTLY_SOLD,471400.0,https://photos.zillowstatic.com/fp/f5b8c1b3a5e...,471900,3.0,...,-1,3.0,2245.0,USA,USD,acres,True,NaN,NaN,NaN
1,1682665200000,SINGLE_FAMILY,0.350000,"132 W Holding Ave, Wake Forest, NC 27587",Sold 04/28/2023,RECENTLY_SOLD,373700.0,https://photos.zillowstatic.com/fp/cc3dddbd058...,375000,3.0,...,-1,3.0,1815.0,USA,USD,acres,True,NaN,NaN,NaN
2,1682665200000,SINGLE_FAMILY,0.429982,"2809 Stirrup Ct, Wake Forest, NC 27587",Sold 04/28/2023,RECENTLY_SOLD,446000.0,https://maps.googleapis.com/maps/api/staticmap...,382500,3.0,...,-1,3.0,2350.0,USA,USD,acres,True,NaN,NaN,NaN
3,1682665200000,SINGLE_FAMILY,0.260000,"900 Finchurch Cir, Wake Forest, NC 27587",Sold 04/28/2023,RECENTLY_SOLD,329700.0,https://photos.zillowstatic.com/fp/c16d597cdf7...,330000,3.0,...,-1,2.0,1332.0,USA,USD,acres,True,NaN,NaN,NaN
4,1682665200000,SINGLE_FAMILY,0.920000,"2812 Kimmon Way, Wake Forest, NC 27587",Sold 04/28/2023,RECENTLY_SOLD,677600.0,https://photos.zillowstatic.com/fp/bf2640f3101...,680000,4.0,...,-1,3.0,2600.0,USA,USD,acres,True,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4303,1641974400000,SINGLE_FAMILY,0.480000,"4 S Nipper Ct, Durham, NC 27704",Sold 01/12/2022,RECENTLY_SOLD,298800.0,https://photos.zillowstatic.com/fp/697f70fb343...,245000,3.0,...,-1,2.0,1310.0,USA,USD,acres,True,NaN,NaN,NaN
4304,1641974400000,SINGLE_FAMILY,8276.400000,"2004 Buffalo Way, Durham, NC 27704",Sold 01/12/2022,RECENTLY_SOLD,301600.0,https://photos.zillowstatic.com/fp/788db1bbbdd...,270000,3.0,...,-1,3.0,1263.0,USA,USD,sqft,True,NaN,NaN,NaN
4305,1641974400000,SINGLE_FAMILY,8276.400000,"3008 Kenan Rd, Durham, NC 27704",Sold 01/12/2022,RECENTLY_SOLD,492800.0,https://photos.zillowstatic.com/fp/98eedd159e7...,470000,3.0,...,-1,3.0,1772.0,USA,USD,sqft,True,NaN,NaN,NaN
4306,1641888000000,SINGLE_FAMILY,0.326000,"703 W Carver St, Durham, NC 27704",Sold 01/11/2022,RECENTLY_SOLD,348300.0,https://photos.zillowstatic.com/fp/1ead45e8823...,332000,3.0,...,-1,2.0,1408.0,USA,USD,acres,True,NaN,NaN,NaN


In [42]:
# If props.AreaUnit is = to sqft then convert it to acres
data.loc[(data['props.lotAreaUnit'] == 'sqft'), 'props.lotAreaValue'] /= 43560
data

,props.dateSold,props.propertyType,props.lotAreaValue,props.address,props.variableData.text,props.variableData.type,props.zestimate,props.imgSrc,props.price,props.bedrooms,...,props.daysOnZillow,props.bathrooms,props.livingArea,props.country,props.currency,props.lotAreaUnit,props.hasImage,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,1682665200000,SINGLE_FAMILY,0.920000,"7525 Matherly Dr, Wake Forest, NC 27587",Sold 04/28/2023,RECENTLY_SOLD,471400.0,https://photos.zillowstatic.com/fp/f5b8c1b3a5e...,471900,3.0,...,-1,3.0,2245.0,USA,USD,acres,True,NaN,NaN,NaN
1,1682665200000,SINGLE_FAMILY,0.350000,"132 W Holding Ave, Wake Forest, NC 27587",Sold 04/28/2023,RECENTLY_SOLD,373700.0,https://photos.zillowstatic.com/fp/cc3dddbd058...,375000,3.0,...,-1,3.0,1815.0,USA,USD,acres,True,NaN,NaN,NaN
2,1682665200000,SINGLE_FAMILY,0.429982,"2809 Stirrup Ct, Wake Forest, NC 27587",Sold 04/28/2023,RECENTLY_SOLD,446000.0,https://maps.googleapis.com/maps/api/staticmap...,382500,3.0,...,-1,3.0,2350.0,USA,USD,acres,True,NaN,NaN,NaN
3,1682665200000,SINGLE_FAMILY,0.260000,"900 Finchurch Cir, Wake Forest, NC 27587",Sold 04/28/2023,RECENTLY_SOLD,329700.0,https://photos.zillowstatic.com/fp/c16d597cdf7...,330000,3.0,...,-1,2.0,1332.0,USA,USD,acres,True,NaN,NaN,NaN
4,1682665200000,SINGLE_FAMILY,0.920000,"2812 Kimmon Way, Wake Forest, NC 27587",Sold 04/28/2023,RECENTLY_SOLD,677600.0,https://photos.zillowstatic.com/fp/bf2640f3101...,680000,4.0,...,-1,3.0,2600.0,USA,USD,acres,True,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4303,1641974400000,SINGLE_FAMILY,0.480000,"4 S Nipper Ct, Durham, NC 27704",Sold 01/12/2022,RECENTLY_SOLD,298800.0,https://photos.zillowstatic.com/fp/697f70fb343...,245000,3.0,...,-1,2.0,1310.0,USA,USD,acres,True,NaN,NaN,NaN
4304,1641974400000,SINGLE_FAMILY,0.190000,"2004 Buffalo Way, Durham, NC 27704",Sold 01/12/2022,RECENTLY_SOLD,301600.0,https://photos.zillowstatic.com/fp/788db1bbbdd...,270000,3.0,...,-1,3.0,1263.0,USA,USD,sqft,True,NaN,NaN,NaN
4305,1641974400000,SINGLE_FAMILY,0.190000,"3008 Kenan Rd, Durham, NC 27704",Sold 01/12/2022,RECENTLY_SOLD,492800.0,https://photos.zillowstatic.com/fp/98eedd159e7...,470000,3.0,...,-1,3.0,1772.0,USA,USD,sqft,True,NaN,NaN,NaN
4306,1641888000000,SINGLE_FAMILY,0.326000,"703 W Carver St, Durham, NC 27704",Sold 01/11/2022,RECENTLY_SOLD,348300.0,https://photos.zillowstatic.com/fp/1ead45e8823...,332000,3.0,...,-1,2.0,1408.0,USA,USD,acres,True,NaN,NaN,NaN


In [43]:
# Extract zip code from address
data['zip'] = data['props.address'].str.extract('(?<=NC )([^,]*)(?=$)', expand = True)
data['zip'].astype(str).astype(int)
data['zip'].head()

0    27587
1    27587
2    27587
3    27587
4    27587
Name: zip, dtype: object

In [44]:
# Extract city code from address
data['city'] = data['props.address'].str.extract('(?<=, )([^,]*)(?=,)', expand = True)   
data['city'].head()

0    Wake Forest
1    Wake Forest
2    Wake Forest
3    Wake Forest
4    Wake Forest
Name: city, dtype: object

In [45]:
# Create lists of cities in each county
durham_city = {'Durham', 'Chapel Hill', 'Creedmoor','Durh'}
wake_cty = {'Apex','Cary','Fuquay Varina','Garner','Holly Springs','Knightdale',
            'Morrisville','Raleigh','Rolesville','Wake Forest','Wendell','Zebulon','Fuquay-varina'}
data['city'].value_counts()

Durham           4305
Raleigh          1978
Cary              757
Apex              438
Garner            317
Wake Forest       280
Knightdale        236
Fuquay Varina     174
Rolesville        120
Creedmoor          80
Holly Springs      79
Morrisville        79
Chapel Hill         2
Fuquay-varina       1
Durh                1
Name: city, dtype: int64

In [46]:
# Create county column
data['county'] = np.where(data['city'].isin(durham_city), 'Durham', 'Wake')
data['county_val'] = np.where(data['city'].isin(durham_city), 0, 1)
data[['city','county','county_val']].sample(20)

,city,county,county_val
199,Durham,Durham,0
3760,Durham,Durham,0
1390,Durham,Durham,0
2436,Durham,Durham,0
1785,Rolesville,Wake,1
3559,Apex,Wake,1
1383,Cary,Wake,1
874,Raleigh,Wake,1
1805,Durham,Durham,0
126,Wake Forest,Wake,1


In [47]:
# Determine how many per county
data['county'].value_counts()

Wake      4460
Durham    4388
Name: county, dtype: int64

In [48]:
# Create data_df dataframe from select columns in data dataframe
data_df = data[['county_val','city','zip','props.zestimate','props.propertyType','props.bedrooms','props.bathrooms','props.livingArea','props.lotAreaValue','props.latitude','props.longitude']]

# Drop rows where bedrooms, bathrooms, or livingArea = 0
data_df = data_df[data_df['props.bedrooms'] != 0]
data_df = data_df[data_df['props.bathrooms'] != 0]
data_df = data_df[data_df['props.livingArea'] != 0]

# Drop NA
data_df = data_df.dropna()

data_df.head()

,county_val,city,zip,props.zestimate,props.propertyType,props.bedrooms,props.bathrooms,props.livingArea,props.lotAreaValue,props.latitude,props.longitude
0,1,Wake Forest,27587,471400.0,SINGLE_FAMILY,3.0,3.0,2245.0,0.920000,35.997555,-78.544770
1,1,Wake Forest,27587,373700.0,SINGLE_FAMILY,3.0,3.0,1815.0,0.350000,35.972195,-78.516880
2,1,Wake Forest,27587,446000.0,SINGLE_FAMILY,3.0,3.0,2350.0,0.429982,35.937810,-78.527550
3,1,Wake Forest,27587,329700.0,SINGLE_FAMILY,3.0,2.0,1332.0,0.260000,35.994972,-78.504776
4,1,Wake Forest,27587,677600.0,SINGLE_FAMILY,4.0,3.0,2600.0,0.920000,36.019070,-78.553570


In [49]:
# Change dtypes of columns
data_df['zip'] = data_df['zip'].astype(str).astype(int)
data_df['props.bedrooms'] = data_df['props.bedrooms'].astype(int)
data_df['props.bathrooms'] = data_df['props.bathrooms'].astype(int)
data_df.dtypes

county_val              int32
city                   object
zip                     int32
props.zestimate       float64
props.propertyType     object
props.bedrooms          int32
props.bathrooms         int32
props.livingArea      float64
props.lotAreaValue    float64
props.latitude        float64
props.longitude       float64
dtype: object

In [50]:
# Describe data_df
data_df.describe()

,county_val,zip,props.zestimate,props.bedrooms,props.bathrooms,props.livingArea,props.lotAreaValue,props.latitude,props.longitude
count,8782.000000,8782.000000,8.782000e+03,8782.000000,8782.000000,8782.000000,8782.000000,8782.000000,8782.000000
mean,0.501822,27634.991004,5.110381e+05,3.437144,2.729333,2187.121726,0.497488,35.888049,-78.794412
std,0.500025,76.116451,2.805756e+05,0.847486,1.019434,1320.521611,1.826194,0.138120,0.147880
min,0.000000,27502.000000,3.220000e+04,1.000000,1.000000,1.000000,0.000000,35.450110,-79.043860
25%,0.000000,27571.000000,3.460000e+05,3.000000,2.000000,1454.250000,0.167000,35.776990,-78.911600
50%,1.000000,27616.000000,4.453000e+05,3.000000,3.000000,1973.500000,0.250000,35.922020,-78.847495
75%,1.000000,27705.000000,5.927000e+05,4.000000,3.000000,2683.000000,0.460000,35.996836,-78.690742
max,1.000000,27713.000000,5.627300e+06,8.000000,10.000000,73140.000000,82.239991,36.145874,-78.401010


In [51]:

# Determine # of unique values per column
data_df.nunique()

county_val               2
city                    15
zip                     30
props.zestimate       4901
props.propertyType       7
props.bedrooms           8
props.bathrooms         10
props.livingArea      3000
props.lotAreaValue    1327
props.latitude        7766
props.longitude       7342
dtype: int64

In [52]:
# categorical columns
categorical_cols = ['props.propertyType']

data_cat_df= pd.get_dummies(data_df, columns = categorical_cols)


In [53]:
data_lg = data_cat_df.drop(['county_val', 'zip', 'props.longitude', 'props.latitude', 'props.propertyType_APARTMENT', 'props.propertyType_CONDO', 'props.propertyType_LOT', 'props.propertyType_MANUFACTURED', 'props.propertyType_MULTI_FAMILY', 'props.propertyType_SINGLE_FAMILY', 'props.propertyType_TOWNHOUSE'], axis=1)


In [54]:
# encode the city 
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
data_lg['city'] = LabelEncoder().fit_transform(data_lg['city'])
X

,city,props.bedrooms,props.bathrooms,props.livingArea,props.lotAreaValue
0,14,3,3,2245.0,0.920000
1,14,3,3,1815.0,0.350000
2,14,3,3,2350.0,0.429982
3,14,3,2,1332.0,0.260000
4,14,4,3,2600.0,0.920000
...,...,...,...,...,...
4303,5,3,2,1310.0,0.480000
4304,5,3,3,1263.0,0.190000
4305,5,3,3,1772.0,0.190000
4306,5,3,2,1408.0,0.326000


In [55]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [56]:
# Define x & y 
X = data_lg[['city', 'props.bedrooms', 'props.bathrooms', 'props.livingArea', 'props.lotAreaValue']]
y = data_lg['props.zestimate']

In [57]:
from sklearn.model_selection import train_test_split

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4, random_state=42)

In [59]:
#shapes of splitted data
print("X_train:",X_train.shape)
print("X_test:",X_test.shape)
print("Y_train:",y_train.shape)
print("Y_test:",y_test.shape)

X_train: (5269, 5)
X_test: (3513, 5)
Y_train: (5269,)
Y_test: (3513,)


In [60]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

In [61]:
# Train the model
model.fit(X_train, y_train)

LinearRegression()

In [62]:
# Calculate the mean_squared_error and the r-squared value
# for the testing data

from sklearn.metrics import mean_squared_error, r2_score

In [63]:
# Use our model to make predictions
predicted = model.predict(X_test)

In [64]:
# Score the predictions with mse and r2
mse = mean_squared_error(y_test, predicted)
r2 = r2_score(y_test, predicted)

In [65]:
print(f"mean squared error (MSE): {mse}")
print(f"R-squared (R2): {r2}")

mean squared error (MSE): 47712655451.19887
R-squared (R2): 0.4805267061992733


In [66]:
# Call the `score()` method on the model to show the R2 score

model.score(X_test, y_test)

0.4805267061992733

In [67]:
# giving inputs to the machine learning model
# features = [[city, bedrooms, bathrooms, livingArea, lotAreaValue]]

features = np.array([[1, 2, 2, 1900, .5]])
# using inputs to predict the output
prediction = model.predict(features)
print("Prediction Price: {}".format(prediction))

Prediction Price: [403528.32439169]


In [68]:
import pickle

In [69]:
# Store model in pickle
pickle.dump(model, open('model.pkl', 'wb'))

In [73]:
# Load pickle model and predict imput values
model = pickle.load(open('model.pkl','rb'))
print(model.predict([[1, 4, 2, 1300, .5]]))

[431260.54915974]
